In [1]:
import sys
sys.path.insert(0, '/iblm/netapp/home/dlaub/projects/hyena-dna/')

from src.dataloaders.gvl import MultiFasta  # noqa: E402

In [2]:
dm = MultiFasta(
    '/iblm/netapp/home/dlaub/projects/hyena-dna/data/gvl_multifasta/32000',
    max_length=32768,
    batch_size=16,
    max_memory_gb=16,
)

RuntimeError: Need name column to use for identifying splits.